In [ ]:
#Read Expression Matrices

In [2]:
readexprs = function(path, lbl){
    
    df = read.csv(path , sep=',')
    rownames(df) = df$X
    df = df[, -1]
    
    names = paste0(lbl, 1:ncol(df))
    colnames(df) = names
    
    return(df)
}

In [7]:
#Column Data

In [8]:
prep = function(case, control){
    
    library(dplyr)
    
    all = cbind(case, control)
    
    coldata_late = data.frame(Sample=colnames(case), Group=rep(c('R'), each=ncol(case)))
    coldata_early = data.frame(Sample=colnames(control), Group=rep(c('S'), each=ncol(control)))
    
    coldata = rbind(coldata_late, coldata_early)
    rownames(coldata) = coldata$Sample
    
    return(coldata)
}

In [14]:
#DESEQ2 DIfferential Expression

In [15]:
de = function(sensitive, resistance, coldata){
    
    library(DESeq2)
    library(dplyr)
    library(devtools)
    library(BiocParallel)
    library(data.table)
    
    register(MulticoreParam())
    
    dataset = cbind(resistance, sensitive)
    
    dds = DESeqDataSetFromMatrix(countData = round(dataset), 
                                 colData = coldata,
                                 design = ~ Group)
    
    dds$Group = relevel(dds$Group, ref = "R")
    
    keep = rowSums(counts(dds)) >= 10
    dds = dds[keep,]
        
    dds_Deseq = DESeq(dds, parallel=TRUE, fitType = "local")
    dds_var = DESeq2::vst(dds, blind = TRUE, fitType = "local")
    
    res = results(dds_Deseq, contrast=c("Group", "R", "S"), alpha=0.05, parallel=TRUE)
    resOrdered = res[order(res$padj), ]
    
    all_genes = as.character(rownames(resOrdered))
    all_genes = sub("\\.\\d+", "", all_genes)
    rownames(resOrdered) = all_genes
    
    up = resOrdered[which(resOrdered$log2FoldChange > 1), ]
    up = up[which(up$padj < 0.05), ]
    up_genes = dataset[rownames(up), ]
    
    keep_up = rowSums(up_genes) >= 10
    up_genes = up_genes[keep_up,]
    up_genes = na.omit(up_genes)
    
    up_genes = DESeq2::vst(up_genes)
    
    return(up_genes)

}

In [ ]:
limma = function(df1, df2){
    
    library(limma)
    library(data.table)
    library(dplyr)
    
    design = model.matrix(~ 0 + factor(c(rep(1, ncol(df1)), rep(2, ncol(df2)))))
    colnames(design) = c('R', 'S')
    
    dataset = cbind(df1, df2) 
    dataset = normalizeBetweenArrays(dataset, method="quantile")
    
    fit = lmFit(dataset, design, method='ls') 
    
    contr = makeContrasts(R - S, levels = colnames(coef(fit)))
    tmp = contrasts.fit(fit, contr)
    
    fit2 = eBayes(tmp, robust=TRUE) 
            
    results = decideTests(fit2, method='hierarchical', adjust.method='BH', p.value=0.05)
    sig = topTable(fit2, n=Inf, adjust='BH', coef=1, sort.by='P', p.value=0.05)
    
    
    up = subset(sig, logFC > 1)
 
    up = df1[rownames(up) , ]
    
    return(up)

}

In [20]:
#Soft-Thresholding

In [21]:
power = function(dataset){
    
    suppressMessages(library(WGCNA))
    
    enableWGCNAThreads(nThreads=8)
    allowWGCNAThreads()
    options(stringsAsFactors = FALSE)
    
    dataset = t(dataset) 
    
    powers = c(c(1:10), seq(from = 12, to=20, by=2))
    sft = pickSoftThreshold(dataset, powerVector = powers, verbose = 5)
    
    options(repr.plot.width=5,repr.plot.height=5,repr.plot.res=200)
    par(mfrow = c(1,2))
    plot(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
         xlab='Soft Threshold (power)', cex.lab=1.2, cex.axis=1.2, cex.main=1.2, cex.sub=1.2,
         ylab='Scale Free Topology Model Fit,signed R^2',type='n',
         main = paste('Scale independence'))
    
    text(sft$fitIndices[, 1], -sign(sft$fitIndices[, 3])*sft$fitIndices[, 2],
         labels=powers,cex=1, col='red')
    abline(h=0.85,col='red')
    
    plot(sft$fitIndices[,1], sft$fitIndices[,5], cex.lab=1.2, cex.axis=1.2, cex.main=1.2, cex.sub=1.2,
         xlab='Soft Threshold (power)',ylab='Mean Connectivity', type='n',
         main = paste('Mean connectivity'))
    
    text(sft$fitIndices[, 1], sft$fitIndices[, 5],
         labels=powers, cex=1, col='red')

}

In [26]:
#Co-Expression Analysis

In [34]:
TOM_Matrices = function(dataset, power, thresh, name){
    
    library(WGCNA)
    
    enableWGCNAThreads(nThreads=12)
    allowWGCNAThreads()
    options(stringsAsFactors = FALSE)
    
    dat = c()
    
    adj = abs(bicor(t(dataset), use = 'pairwise.complete.obs'))^power
    dissTOM = TOMdist(adj, TOMType = "signed")
    hierTOMa = hclust(as.dist(dissTOM), method='average')
    
    Gene_Modules = labels2colors(cutreeDynamic(hierTOMa, method='tree', cutHeight=0.99))
    colors = unique(Gene_Modules)
    
    datME = moduleEigengenes(t(dataset),Gene_Modules)$eigengenes
    datKME = signedKME(t(dataset), datME, outputColumnName='')
        
    intModules = table(Gene_Modules)
    intModules = as.data.frame(intModules)
    intModules =intModules$Gene_Modules
    intModules = as.character(intModules)
    
    for (color in intModules){
        
        FilterGenes = abs(subset(datKME, select=c(color))) > thresh
        genes = dimnames(data.frame(t(dataset)))[[2]][FilterGenes]
        
        dat = append(dat, genes)
    
    }
    
    dat = unique(dat)
    dataset = dataset[dat, ]
    
    adj = abs(bicor(t(dataset), use = 'pairwise.complete.obs'))^power
    dissTOM = TOMdist(adj, TOMType = "signed")
    
    write.csv(dissTOM, paste('/directory/', name, sep=''))

}
